### CODE FROM
https://github.com/smortezavi/Randomized_SVD_GPU/blob/master/pytorch_randomized_svd.ipynb

In [1]:
import numpy as np
import timeit
import pandas as pd
from sklearn import decomposition
#import fbpca
import torch
from scipy import linalg

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
def simple_randomized_svd(M, k=10):
    m, n = M.shape
    transpose = False
    if m < n:
        transpose = True
        M = M.T
    rand_matrix = np.random.normal(size=(M.shape[1], k))  # short side by k
    Q, _ = np.linalg.qr(M @ rand_matrix, mode='reduced')  # long side by k
    smaller_matrix = Q.T @ M                              # k by short side
    U_hat, s, V = np.linalg.svd(smaller_matrix, full_matrices=False)
    U = Q @ U_hat
    
    if transpose:
        return V.T, s.T, U.T
    else:
        return U, s, V

In [ ]:
def simple_randomized_torch_svd(M, k=10):
    B = torch.tensor(M).cuda(0)
    m, n = B.size()
    transpose = False
    if m < n:
        transpose = True
        B = B.transpose(0, 1).cuda(0)
        m, n = B.size()
    rand_matrix = torch.rand((n,k), dtype=torch.double).cuda(0)  # short side by k
    Q, _ = torch.qr(B @ rand_matrix)                              # long side by k
    Q.cuda(0)
    smaller_matrix = (Q.transpose(0, 1) @ B).cuda(0)             # k by short side
    U_hat, s, V = torch.svd(smaller_matrix,False)
    U_hat.cuda(0)
    U = (Q @ U_hat)
    
    if transpose:
        return V.transpose(0, 1), s, U.transpose(0, 1)
    else:
        return U, s, V

In [1]:
# computes an orthonormal matrix whose range approximates the range of A
# power_iteration_normalizer can be safe_sparse_dot (fast but unstable), LU (imbetween), or QR (slow but most accurate)
def randomized_range_finder(A, size, n_iter=5, device = device):
    A = A.to(device)
    Q = torch.tensor(np.random.normal(size=(A.shape[1], size))).to(device)
    
    for i in range(n_iter):
        P,L, _ = torch.linalg.lu(A @ Q)
        Q = P @ L
        P, L, _ = torch.linalg.lu(A.T @ Q)
        Q = P @ L
        
    Q, _ = linalg.qr(A @ Q, mode='economic')
    return Q

NameError: name 'device' is not defined

In [7]:
def randomized_svd(M, n_components, n_oversamples=10, n_iter=4):
    n_random = n_components + n_oversamples
    
    M = torch.tensor(M)#.to(device)
    Q = torch.tensor(randomized_range_finder(M, n_random, n_iter))#.to(device)
    # project M to the (k + p) dimensional space using the basis vectors
    M = torch.tensor(M)#.to(device)
    B = Q.transpose(0, 1) @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = torch.linalg.svd(B, full_matrices=False)
    Uhat, s, V = Uhat#.to(device), s#.to(device), V#.to(device)
    Uhat = torch.tensor(Uhat).cuda(0)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [ ]:
def randomized_svd_original(M, n_components, n_oversamples=10, n_iter=4):
    
    n_random = n_components + n_oversamples
    
    Q = randomized_range_finder(M, n_random, n_iter)
    # project M to the (k + p) dimensional space using the basis vectors
    B = Q.T @ M
    # compute the SVD on the thin matrix: (k + p) wide
    Uhat, s, V = torch.linalg.svd(B, full_matrices=False)
    del B
    U = Q @ Uhat
    
    return U[:, :n_components], s[:n_components], V[:n_components, :]

In [8]:
A = torch.tensor(np.random.uniform(-40,40,[10,100]))
randomized_svd(A, 5)

/tmp/ipykernel_3787052/4071521345.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  M = torch.tensor(M)#.to(device)


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
A=torch.tensor(A).to(device)

In [ ]:
A = np.random.uniform(-40,40,[10,100]) 
simple_randomized_svd(A)

In [ ]:
A = np.random.uniform(-40,40,[10,100])
simple_randomized_torch_svd(A)

In [7]:
A = np.random.uniform(-1,1,[200 * 200,50000])

In [ ]:
P,L,U2 = linalg.lu(A)

In [ ]:
P@L

In [ ]:
Lp, U = linalg.lu(A,permute_l=True)

In [ ]:
print(Lp)

In [8]:
linalg.qr(A)#,mode='economic')

TypeError: _ArrayMemoryError.__init__() missing 1 required positional argument: 'dtype'

: 

In [ ]:
linalg.qr(A,mode='economic')

In [ ]:
Atensor = torch.tensor(A).to(device)

In [ ]:
Atensor.shape

In [ ]:
torch.linalg.qr(Atensor)